In [1]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Stats/Regresions Packages
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

# make sure charts appear in the notebook:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [17]:
file1 = 'C:/Users/Adeel/Desktop/Dataset econometrics/2010/Call1Q10.xlsx'
df = pd.read_excel(file1)


In [18]:
df.info

<bound method DataFrame.info of            Date  Cash And Balances Due From Depository Institutions  \
0      20100331                                             7374.0    
1      20100331                                             1897.0    
2      20100331                                             6706.0    
3      20100331                                              186.0    
4      20100331                                             1082.0    
5      20100331                                            23889.0    
6      20100331                                            22596.0    
7      20100331                                             2485.0    
8      20100331                                            34827.0    
9      20100331                                          5183081.0    
10     20100331                                             2561.0    
11     20100331                                             7458.0    
12     20100331                              

In [19]:
df.shape

(27740, 97)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27740 entries, 0 to 27739
Data columns (total 97 columns):
Date                                                                                                                                     27740 non-null int64
Cash And Balances Due From Depository Institutions                                                                                       27704 non-null float64
Interest-Bearing Balances                                                                                                                26833 non-null float64
Noninterest-Bearing Balances And Currency And Coin                                                                                       26834 non-null float64
Total Loans And Leases, Gross                                                                                                            27619 non-null float64
Available-For-Sale Securities, Total                                                         

In [21]:
df.dropna

<bound method DataFrame.dropna of            Date  Cash And Balances Due From Depository Institutions  \
0      20100331                                             7374.0    
1      20100331                                             1897.0    
2      20100331                                             6706.0    
3      20100331                                              186.0    
4      20100331                                             1082.0    
5      20100331                                            23889.0    
6      20100331                                            22596.0    
7      20100331                                             2485.0    
8      20100331                                            34827.0    
9      20100331                                          5183081.0    
10     20100331                                             2561.0    
11     20100331                                             7458.0    
12     20100331                            

In [22]:
df.shape

(27740, 97)

In [23]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Date,27740.0,2.010078e+07,3.355231e+02,20100331.00,20100331.00,20100630.00,2.010093e+07,2.010123e+07
Cash And Balances Due From Depository Institutions,27704.0,1.893124e+05,3.085138e+06,0.00,3800.00,9243.00,2.376275e+04,1.675770e+08
Interest-Bearing Balances,26833.0,1.207750e+05,2.533285e+06,0.00,466.00,4364.00,1.573700e+04,1.458750e+08
Noninterest-Bearing Balances And Currency And Coin,26834.0,2.985544e+04,5.534996e+05,0.00,1505.00,3121.00,6.536250e+03,3.188000e+07
"Total Loans And Leases, Gross",27619.0,9.972819e+05,1.567801e+07,0.00,42041.00,97275.00,2.266090e+05,7.416214e+08
"Available-For-Sale Securities, Total",27610.0,3.192473e+05,6.232427e+06,0.00,6688.50,21699.00,5.658175e+04,3.356920e+08
Unearned Income On Loans,27611.0,5.288828e+02,1.265127e+04,0.00,0.00,0.00,3.000000e+00,1.053000e+06
Premises And Fixed Assets (Including Capitalized Leases),26830.0,1.622108e+04,2.174251e+05,0.00,704.00,2391.00,6.229750e+03,1.158983e+07
Other Assets,26838.0,1.034208e+05,2.182314e+06,0.00,1471.00,3609.00,9.603000e+03,1.293896e+08
Allowance For Loan And Lease Losses,26828.0,3.402117e+04,6.356725e+05,0.00,592.00,1499.00,3.951500e+03,2.563366e+07


In [24]:
df.isnull().sum()

Date                                                                                                                                        0
Cash And Balances Due From Depository Institutions                                                                                         36
Interest-Bearing Balances                                                                                                                 907
Noninterest-Bearing Balances And Currency And Coin                                                                                        906
Total Loans And Leases, Gross                                                                                                             121
Available-For-Sale Securities, Total                                                                                                      130
Unearned Income On Loans                                                                                                                  129
Premis

In [26]:
df.corr()

,Date,Cash And Balances Due From Depository Institutions,Interest-Bearing Balances,Noninterest-Bearing Balances And Currency And Coin,"Total Loans And Leases, Gross","Available-For-Sale Securities, Total",Unearned Income On Loans,Premises And Fixed Assets (Including Capitalized Leases),Other Assets,Allowance For Loan And Lease Losses,...,Total Investment Securities - Market Value,Tier 2 (Supplementary) Capital.1,Tier 1 Leverage Capital Ratio.1,Tier 1 Risk-Based Capital Ratio.1,Total Risk-Based Capital Ratio.1,Other Assets - Totals,Net Income (Loss) Attributable To Bank And Noncontrolling Minority Interests,Total Equity Capital Including Noncontrolling Minority Interests In Consolidated Subsidiaries,Primary Aba Number,Fdic Certificate Id
Date,1.000000,-0.002250,-0.003979,-0.002452,-0.002231,-0.000848,-0.001132,-0.001918,-0.002924,-0.004179,...,0.008597,-0.002870,0.005793,0.008273,0.008294,0.003469,0.018707,-0.001478,-0.002086,-0.002340
Cash And Balances Due From Depository Institutions,-0.002250,1.000000,0.993043,0.846968,0.770572,0.813350,0.511950,0.766619,0.893708,0.751993,...,0.549779,0.760409,-0.008898,0.002329,0.002818,0.528782,0.692633,0.879785,-0.051845,-0.037389
Interest-Bearing Balances,-0.003979,0.993043,1.000000,0.784338,0.760037,0.813166,0.542366,0.711319,0.853051,0.704379,...,0.583245,0.696022,-0.007420,0.000082,0.000553,0.521112,0.635770,0.832957,-0.037553,-0.024272
Noninterest-Bearing Balances And Currency And Coin,-0.002452,0.846968,0.784338,1.000000,0.907386,0.948759,0.368393,0.891632,0.924608,0.831685,...,0.072470,0.915419,-0.014030,0.012188,0.012672,0.162023,0.825656,0.932426,-0.038250,-0.025217
"Total Loans And Leases, Gross",-0.002231,0.770572,0.760037,0.907386,1.000000,0.893892,0.296434,0.949558,0.943871,0.926503,...,0.281944,0.943864,-0.013117,-0.004291,-0.003659,0.951029,0.812347,0.979536,-0.029533,-0.027894
"Available-For-Sale Securities, Total",-0.000848,0.813350,0.813166,0.948759,0.893892,1.000000,0.340384,0.921697,0.959671,0.810043,...,0.977536,0.932718,-0.013119,-0.003062,-0.002512,0.295289,0.811033,0.946984,-0.036371,-0.029937
Unearned Income On Loans,-0.001132,0.511950,0.542366,0.368393,0.296434,0.340384,1.000000,0.213221,0.366999,0.309733,...,0.072982,0.176079,-0.006065,-0.002017,-0.001936,0.020424,0.302396,0.357978,-0.024339,-0.018081
Premises And Fixed Assets (Including Capitalized Leases),-0.001918,0.766619,0.711319,0.891632,0.949558,0.921697,0.213221,1.000000,0.931026,0.832892,...,0.390245,0.952080,-0.017273,-0.005176,-0.004514,0.555477,0.770984,0.950910,-0.030793,-0.036216
Other Assets,-0.002924,0.893708,0.853051,0.924608,0.943871,0.959671,0.366999,0.931026,1.000000,0.874752,...,0.350142,0.939473,-0.009677,-0.002970,-0.002401,0.972760,0.786254,0.977671,-0.035435,-0.029215
Allowance For Loan And Lease Losses,-0.004179,0.751993,0.704379,0.831685,0.926503,0.810043,0.309733,0.832892,0.874752,1.000000,...,0.040347,0.853242,-0.010485,-0.003842,-0.003208,0.850609,0.739506,0.907386,-0.033253,-0.021110
